In [1]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric
from torch_geometric.datasets import CoraFull, Planetoid, CitationFull
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric.nn as gnn 

from models import GAT, GraphSAGE, GIN
from utils import train_model, test_model
from mean_average_distance import MAD, MADGap

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

SAVE_PATH = 'results'
LR = 0.01

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/jaewoo/anaconda3/envs/torch_tutorial/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


device(type='cuda')

In [2]:
dataset = CitationFull(root='dataset/Cora', name='Cora', transform=NormalizeFeatures())

In [3]:
data = dataset[0]
df = pd.DataFrame(data.x)
df['y'] = data.y
train, valid = train_test_split(df, stratify=df.y, test_size=0.33)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[train.index]=True

In [6]:
sage = GraphSAGE(in_channels=dataset.num_features, hidden_channels=256, number_of_classes=dataset.num_classes, num_of_hidden_layers=4, device=device)

models = [sage]

for model in models:
    print(f'model: {model.name}, params: {model.num_of_parameters}')


model: GraphSAGE, params: 4871750


In [7]:
model_losses = []
model_accs = []
for model in models:
    print(f'Model: {model.name} | Number of parameters: {model.get_n_params()}')
    model = model.to(device)
    data = data.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=5e-4)
    losses = []
    accs = []
    for epoch in range(500):
        loss, acc = train_model(model, data, optimizer, criterion)
        losses.append(loss.item())
        accs.append(100*acc)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {100*acc:.2f}')
    model_losses.append(losses)
    model_accs.append(accs)
    report = test_model(model, data)
    result = pd.DataFrame(report).T
    result_sliced = result.iloc[:-3 if len(result) < 23 else 20, :]
    acc = result.loc['accuracy'][0]
    result.loc['minorities-f1',:] = result_sliced.mean(axis=0)
    result.to_csv(os.path.join(SAVE_PATH, f'{model.name}_layers{model.num_of_hidden_layers}_neurons{model.hidden_channels}'+'.csv'))
    print(f'Test Acc: {100*acc}')
    print('==========================================', end='\n\n')

Model: GraphSAGE | Number of parameters: 4871750
Epoch: 000, Loss: 4.2495, Acc: 1.08
Epoch: 001, Loss: 4.0929, Acc: 4.46
Epoch: 002, Loss: 7.1297, Acc: 4.15
Epoch: 003, Loss: 4.0880, Acc: 3.17
Epoch: 004, Loss: 4.2078, Acc: 3.97
Epoch: 005, Loss: 4.4124, Acc: 0.15
Epoch: 006, Loss: 4.0325, Acc: 5.80
Epoch: 007, Loss: 4.5174, Acc: 7.81
Epoch: 008, Loss: 5.6008, Acc: 2.87
Epoch: 009, Loss: 6.4055, Acc: 1.97
Epoch: 010, Loss: 4.1843, Acc: 8.59
Epoch: 011, Loss: 4.7512, Acc: 5.29
Epoch: 012, Loss: 4.7891, Acc: 2.88
Epoch: 013, Loss: 4.4054, Acc: 3.07
Epoch: 014, Loss: 4.2688, Acc: 6.79
Epoch: 015, Loss: 4.1311, Acc: 8.08
Epoch: 016, Loss: 3.7512, Acc: 11.28
Epoch: 017, Loss: 3.6815, Acc: 13.45
Epoch: 018, Loss: 3.5225, Acc: 13.51
Epoch: 019, Loss: 3.4364, Acc: 14.27
Epoch: 020, Loss: 3.2931, Acc: 16.54
Epoch: 021, Loss: 3.2261, Acc: 16.36
Epoch: 022, Loss: 3.2263, Acc: 16.78
Epoch: 023, Loss: 3.1255, Acc: 16.62
Epoch: 024, Loss: 3.0416, Acc: 19.15
Epoch: 025, Loss: 3.0057, Acc: 20.31
Epoch

/home/jaewoo/anaconda3/envs/torch_tutorial/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jaewoo/anaconda3/envs/torch_tutorial/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jaewoo/anaconda3/envs/torch_tutorial/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [8]:
# MAD and MADGap Usage Example
mad = MAD(device=device, global_flag=True)
result = model(dataset.data.x.to(device), dataset.data.edge_index.to(device)).cpu()
print(mad(result))
madgap = MADGap(device, 3, 8)
print(madgap(result, dataset.data.edge_index))

/home/jaewoo/anaconda3/envs/torch_tutorial/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


tensor(0.7056, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3781, device='cuda:0', grad_fn=<SubBackward0>)
